In [29]:
# 神经网络可以通过 torch.nn 包来构建。
# 现在对于自动梯度(autograd)有一些了解，神经网络是基于自动梯度 (autograd)来定义一些模型。
# 一个 nn.Module 包括层和一个方法 forward(input) 它会返回输出(output)。

# 一个典型的神经网络训练过程包括以下几点：
# 1.定义一个包含可训练参数的神经网络
# 2.迭代整个输入
# 3.通过神经网络处理输入
# 4.计算损失(loss)
# 5.反向传播梯度到神经网络的参数
# 6.更新网络的参数，典型的用一个简单的更新方法：weight = weight - learning_rate *gradient

# 定义神经网络
import torch
import torch.nn as nn
import torch.nn.functional as F

# 下面的网络结构如下，用于手写数字识别
# 输入是32*32的灰度图
# 第一层是6通道的5*5的卷积，输出是6通道的28*28(28= 32+1-5), 激活函数是Relu
# 然后做2*2的max pooling，变为6通道的14*14
# 第二层做16通道的5*5，输出是16通道的10*10(10=14+1-5),激活函数是Relu
# 然后继续做2*2的max pooling，变为16通道的5*5
# 然后接全连接层，将16*5*5的输入转变为120维的向量, 激活函数是Relu
# 再接全连接层，将120维向量转为84维, 激活函数是Relu为什么是84？
# 再接全连接层，将84维转为10维，这样就可以直接softmax输出了

class Net(nn.Module):
    def __init__(self):
        # 调用基类初始化函数
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        # nn.Conv2d还可以指定stride
        self.conv1 = nn.Conv2d(1, 6, 5) # 1:input channel, 6:output channel, 5: 5*5 conv kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # 6:input channel, 16:output channel, 5: 5*5 conv kernel
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #16*5*5：input dimension, 120:output dimension
        self.fc2 = nn.Linear(120, 84) #120：input dimension, 84:output dimension
        self.fc3 = nn.Linear(84, 10) #84：input dimension, 10:output dimension，然后直接softmax就可以输出了

    def forward(self, x):
        print('origin x.size():', x.size())
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # 如果第二个参数size是个正方形，直接指定正方形的边长也行
        #self.num_flat_features(x)将16通道5*5的卷积输出展开为一维，第一个-1表示根据第二个维度决定x的size
        x = x.view(-1, self.num_flat_features(x)) 
        #接着几个全连接层+Relu
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    # 实际上x是4维的，第一个维度是batch size，第二个维度是通道channel, 第三个第四个是图像的长宽
    # 下面的函数实际返回的是：通道数*长*宽
    # x.size()[1:] 获得是x.size()除去第一维后剩下的部分
    # for s in size, size应该类似tuple,可以这样遍历其每一个维度的值
    def num_flat_features(self, x):
        #print('1 x.size():',x.size())
        size = x.size()[1:]  # all dimensions except the batch dimension
        #print('2 size:',size)
        num_features = 1
        for s in size:
            num_features *= s
        #print('num_features:',num_features)
        return num_features


net = Net()
print(net)

# 刚定义了一个前馈函数Net，然后反向传播函数被自动通过 autograd 定义了。
# 你可以使用任何张量操作在前馈函数上。

# 一个模型可训练的参数可以通过调用 net.parameters() 返回
params = net.parameters()
#print(params)
params = list(params)
#print(params)
print(len(params))
for x in range(0, len(params)):
    print(x, ':', params[x].size())

# 上面输出了模型的各个层的参数的size，结果如下
# 0 : torch.Size([6, 1, 5, 5]) # 6是输出通道数，1应该是输入的通道?
# 1 : torch.Size([6]) #这个是啥呢？应该是conv1的偏置
# 2 : torch.Size([16, 6, 5, 5])  # 16是输出通道数，6应该是输入的通道，即上层的输出通道数
# 3 : torch.Size([16])
# 4 : torch.Size([120, 400])  # 全连接层，将400维向量映射为120层
# 5 : torch.Size([120])  # 这个是bias么？？？
# 6 : torch.Size([84, 120])
# 7 : torch.Size([84])
# 8 : torch.Size([10, 84])
# 9 : torch.Size([10])


#print(params[0].size()) #输出torch.Size([6, 1, 5, 5]) 即 conv1's .weight，为啥不是[1,6,5,5],那个1代表什么？
#print(params[1].size())

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
0 : torch.Size([6, 1, 5, 5])
1 : torch.Size([6])
2 : torch.Size([16, 6, 5, 5])
3 : torch.Size([16])
4 : torch.Size([120, 400])
5 : torch.Size([120])
6 : torch.Size([84, 120])
7 : torch.Size([84])
8 : torch.Size([10, 84])
9 : torch.Size([10])


In [30]:
# 生成一个随机的输入, [1 1 32 32] 第一个1表示batch size，第二个1是输入的通道，后面是图片的size 32*32
# 所以同时输入4张图片应该是[4 1 32 32]，而输入[1 4 32 32]会包错，说输入了4个channel而期望输入1个channel
input = torch.randn(2, 1, 32, 32)
out = net(input)
print(out)

origin x.size(): torch.Size([2, 1, 32, 32])
tensor([[-0.1257, -0.1190,  0.0738,  0.1356,  0.0118, -0.0110,  0.1030,  0.0886,
          0.0842,  0.0526],
        [-0.1269, -0.1179,  0.0824,  0.1286,  0.0080, -0.0108,  0.1065,  0.0915,
          0.0899,  0.0535]], grad_fn=<AddmmBackward>)


In [31]:
# 把所有参数梯度缓存器置零，用随机的梯度来反向传播
# 参考这篇文章： https://blog.csdn.net/witnessai1/article/details/79763596
# 关于backward和Jacobian矩阵
# 至少知道out.backward()传入的参数的size和out的size应该一样
net.zero_grad()
out.backward(torch.randn(2, 10)) #反向传播

In [37]:
# 还剩下 1）计算损失 2）更新网络中的权重

# 损失函数
# 一个损失函数需要一对输入：模型输出和目标，然后计算一个值来评估输出距离目标有多远。
# 有一些不同的损失函数在 nn 包中。一个简单的损失函数就是 nn.MSELoss ，这计算了均方误差。

output = net(input)
print('output size:', output.size())

target = torch.randn((2, 10))  # a dummy target, for example
print('1 target size:', target.size())
target = target.view(2, -1)  # make it the same shape as output
print('2 target size:', target.size()) # 上面生成的size是[10], 这里target.view(1, -1)函数将其变成2维的[1, 10]

criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)
print(loss.grad_fn)

origin x.size(): torch.Size([2, 1, 32, 32])
output size: torch.Size([2, 10])
1 target size: torch.Size([2, 10])
2 target size: torch.Size([2, 10])
tensor(1.2460, grad_fn=<MseLossBackward>)


In [38]:
# 现在，如果你跟随损失到反向传播路径，可以使用它的 .grad_fn 属性，你将会看到一个这样的计算图：
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#      -> view -> linear -> relu -> linear -> relu -> linear
#      -> MSELoss
#      -> loss

# 所以，当我们调用 loss.backward()，整个图都会微分，
# 而且所有的在图中的requires_grad=True 的张量将会让他们的 grad 张量累计梯度。

print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [39]:
# 为了实现反向传播损失，我们所有需要做的事情仅仅是使用 loss.backward()。
# 你需要清空现存的梯度，要不然帝都将会和现存的梯度累计到一起。
# 现在我们调用 loss.backward() ，然后看一下 con1 的偏置项在反向传播之前和之后的变化。

# zeroes the gradient buffers of all parameters
net.zero_grad() 

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)  # conv1.bias是conv1的偏置变量

loss.backward()  # 反向传播计算梯度

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0120, -0.0053,  0.0040, -0.0026, -0.0049, -0.0063])


In [40]:
# 更新梯度，最简单就是随机梯度下降： weight = weight - learning_rate * gradient
# 例如：
# learning_rate = 0.01
# for f in net.parameters():
#    f.data.sub_(f.grad.data * learning_rate)

# 但如果想用SGD Adam RMSProp，使用torch.optim就好了
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01) # 将optimizer网络的参数关联起来，并设置学习率

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input) # 输入，并计算输出
loss = criterion(output, target) # 计算损失
loss.backward() # 反向传播并计算梯度
optimizer.step()    # Does the update，更新参数

origin x.size(): torch.Size([2, 1, 32, 32])
